# Packages & Includes

In [1]:
using Plots
using StatsBase
using LinearAlgebra
using Statistics
using JLD2
using Dates
using LsqFit
using StaticArrays

# data structure module (needs to be module to avoid overwriting local variables in REPL scope)
include("SpinSimParams.jl")
using .SpinSimParams

# function libraries
include("spin_sims.jl");

# Basic Parameters

In [2]:
# gamma
γ = 2*π*1e6;

# time variables
τ = 100e-6;
dt = 2;

# interaction
α = 0.13;
ω = 0;

# spin ensemble
ν0 = 10;
bw = 0.5;
nfreq = 400;

# initial conditions
dim = 2;
ψ_0 = @SArray [1 0];
ρ_temp = [ψ_0[i]*ψ_0[j] for i = 1:dim, j = 1:dim];
ρ0 = @SMatrix [ρ_temp[1,1] ρ_temp[1,2]; ρ_temp[2,1] ρ_temp[2,2]];

# Lattice/Spatial Dist. Parameters

In [8]:
# lattice parameters
hlk = [1; 1];
θ = [π/2];

# wavevectors
ky = collect(exp10.(LinRange(-3, 0, 20)))
k = ();
for i = 1:size(ky,1)
    k = (k..., [0; ky[i]]);
end

# make the parameter file
params = make_params(γ, τ, dt, α, ω, ν0, bw, nfreq, ρ0);

# load lattice parameters
params["hlk"] = hlk;
params["θ"] = θ;
params["k"] = k;

# create lattice
r, idx_r = make_lattice(params);

# each cluster has a single position
I = ();
R = ();
for i = 1:400
    temp = sample(1:params["nfreq"], params["nfreq"], replace = false);
    I = (I..., temp);
    R = (R..., r[temp,:]);
end

# load the parameters
params["idx_r"] = I;
params["r"] = R;

# set the variables to loop over
vars = ["k", "r"];
params["vars"] = vars;

# Run the Simulation (with spatial distribution)

In [16]:
I, d = make_idx(vars, params)
M_list = Array{Any}(undef, d)

@time begin
    for i in I

        # generate temporary parameters
        tparams = make_temp_params(params, vars, i)

        # simulate
        M_list[i] = spin_echo_sim(tparams)

    end
end

@save "004_M_list.jld2" M_list
@save "004_parameters.jld2" params

┌ Warning: saved type NTuple{400,Array{Float64,2}} is missing field 145 in workspace type; reconstructing
└ @ JLD2 C:\Users\charl\.julia\packages\JLD2\2W9VX\src\data.jl:402


JLD2.ReconstructedTypes.var"##NTuple{400,Array{Float64,2}}#263"
